In [5]:
pip install pandas geopandas matplotlib requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 18.7 MB/s eta 0:00:00m eta 0:00:010:0101
  Using cached geopandas-0.12.2-py3-none-any.whl (1.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 21.8 MB/s eta 0:00:00m eta 0:00:010:01
  Using cached requests-2.29.0-py3-none-any.whl (62 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 19.6 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.6 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 18.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 17.6 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 11.0 MB/s eta 0:00:00
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.39.3-py3-none-

In [7]:
provinces = [{"code":"10","desc":"กรุงเทพมหานคร"},{"code":"11","desc":"สมุทรปราการ"},{"code":"12","desc":"นนทบุรี"},{"code":"13","desc":"ปทุมธานี"},{"code":"14","desc":"พระนครศรีอยุธยา"},{"code":"15","desc":"อ่างทอง"},{"code":"16","desc":"ลพบุรี"},{"code":"17","desc":"สิงห์บุรี"},{"code":"18","desc":"ชัยนาท"},{"code":"19","desc":"สระบุรี"},{"code":"20","desc":"ชลบุรี"},{"code":"21","desc":"ระยอง"},{"code":"22","desc":"จันทบุรี"},{"code":"23","desc":"ตราด"},{"code":"24","desc":"ฉะเชิงเทรา"},{"code":"25","desc":"ปราจีนบุรี"},{"code":"26","desc":"นครนายก"},{"code":"27","desc":"สระแก้ว"},{"code":"30","desc":"นครราชสีมา"},{"code":"31","desc":"บุรีรัมย์"},{"code":"32","desc":"สุรินทร์"},{"code":"33","desc":"ศรีสะเกษ"},{"code":"34","desc":"อุบลราชธานี"},{"code":"35","desc":"ยโสธร"},{"code":"36","desc":"ชัยภูมิ"},{"code":"37","desc":"อำนาจเจริญ"},{"code":"38","desc":"บึงกาฬ"},{"code":"39","desc":"หนองบัวลำภู"},{"code":"40","desc":"ขอนแก่น"},{"code":"41","desc":"อุดรธานี"},{"code":"42","desc":"เลย"},{"code":"43","desc":"หนองคาย"},{"code":"44","desc":"มหาสารคาม"},{"code":"45","desc":"ร้อยเอ็ด"},{"code":"46","desc":"กาฬสินธุ์"},{"code":"47","desc":"สกลนคร"},{"code":"48","desc":"นครพนม"},{"code":"49","desc":"มุกดาหาร"},{"code":"50","desc":"เชียงใหม่"},{"code":"51","desc":"ลำพูน"},{"code":"52","desc":"ลำปาง"},{"code":"53","desc":"อุตรดิตถ์"},{"code":"54","desc":"แพร่"},{"code":"55","desc":"น่าน"},{"code":"56","desc":"พะเยา"},{"code":"57","desc":"เชียงราย"},{"code":"58","desc":"แม่ฮ่องสอน"},{"code":"60","desc":"นครสวรรค์"},{"code":"61","desc":"อุทัยธานี"},{"code":"62","desc":"กำแพงเพชร"},{"code":"63","desc":"ตาก"},{"code":"64","desc":"สุโขทัย"},{"code":"65","desc":"พิษณุโลก"},{"code":"66","desc":"พิจิตร"},{"code":"67","desc":"เพชรบูรณ์"},{"code":"70","desc":"ราชบุรี"},{"code":"71","desc":"กาญจนบุรี"},{"code":"72","desc":"สุพรรณบุรี"},{"code":"73","desc":"นครปฐม"},{"code":"74","desc":"สมุทรสาคร"},{"code":"75","desc":"สมุทรสงคราม"},{"code":"76","desc":"เพชรบุรี"},{"code":"77","desc":"ประจวบคีรีขันธ์"},{"code":"80","desc":"นครศรีธรรมราช"},{"code":"81","desc":"กระบี่"},{"code":"82","desc":"พังงา"},{"code":"83","desc":"ภูเก็ต"},{"code":"84","desc":"สุราษฎร์ธานี"},{"code":"85","desc":"ระนอง"},{"code":"86","desc":"ชุมพร"},{"code":"90","desc":"สงขลา"},{"code":"91","desc":"สตูล"},{"code":"92","desc":"ตรัง"},{"code":"93","desc":"พัทลุง"},{"code":"94","desc":"ปัตตานี"},{"code":"95","desc":"ยะลา"},{"code":"96","desc":"นราธิวาส"}]

In [23]:
import requests
import json

def get_districts(province_code):
    url = 'https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php?API=/api/stat/statcenter/v1/list?action=2'
    payload = {
        'cc':province_code
    }
    response = requests.get(url, params=payload)
    json_response = response.json()
#     json_string = json.dumps(json_response, indent=2, ensure_ascii=False)
#     print(json_string)
    return json_response

def get_subdistricts(district_code):
    res = requests.get('https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php?API=/api/stat/statcenter/v1/list?action=614',{
        'rcode': district_code,
        'yymm': '6501' # need to send this stupid shit
    })
    aa = res.json()[0]['aaCode']
    print(aa)
    url = 'https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php?API=/api/stat/statcenter/v1/list?action=616'
    payload = {
        'yymm': 6501, 
        'rcode': district_code,
        'aa':aa
    }
    response = requests.get(url, params=payload)
    json_response = response.json()
#     json_string = json.dumps(json_response, indent=2, ensure_ascii=False)
#     print(json_string)
    return json_response
  
def get_population_statistics_province_view(start, end):
    url = 'https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php?API=/api/statpophouse/v1/statpop/list?action=41'
    payload = {
        'yymmBegin': start, 
        'yymmEnd': end, 
        'statType':0, 
        'statSubType':999, 
        'subType':99
    }
    response = requests.get(url, params=payload)
    json_response = response.json()
#     json_string = json.dumps(json_response, indent=2, ensure_ascii=False)
#     print(json_string)
    return json_response

def get_population_statistics_district_view(start, end,province_code,district_code):
    url = 'https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php?API=/api/statpophouse/v1/statpop/list?action=44'
    payload = {
        'yymmBegin': start, 
        'yymmEnd': end, 
        'statType':0, 
        'statSubType':999, 
        'subType':99,
        'cc':province_code,
        'rcode': district_code
    }
    response = requests.get(url, params=payload)
    json_response = response.json()
#     json_string = json.dumps(json_response, indent=2, ensure_ascii=False)
#     print(json_string)
    return json_response

def get_population_statistics_subdistrict_view(start, end,province_code,district_code,subdistrict):
    url = 'https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php?API=/api/statpophouse/v1/statpop/list?action=45'
    payload = {
        'yymmBegin': start, 
        'yymmEnd': end, 
        'statType':0, 
        'statSubType':999, 
        'subType':99,
        'cc':province_code,
        'rcode': district_code,
        'tt': subdistrict
    }
    response = requests.get(url, params=payload)
    json_response = response.json()
#     json_string = json.dumps(json_response, indent=2, ensure_ascii=False)
#     print(json_string)
    return json_response
  

In [9]:
bangkok_code = provinces[0]['code']
bkk_districts = get_districts(bangkok_code)
district_code = bkk_districts[0]['lsrcodecode']
bkk_district_1_subdistricts = get_subdistricts(district_code)
print(bkk_district_1_subdistricts)

[
  {
    "lsrcodecode": "1001",
    "lsrcodedesc": "เขตพระนคร"
  },
  {
    "lsrcodecode": "1002",
    "lsrcodedesc": "เขตดุสิต"
  },
  {
    "lsrcodecode": "1003",
    "lsrcodedesc": "เขตหนองจอก"
  },
  {
    "lsrcodecode": "1004",
    "lsrcodedesc": "เขตบางรัก"
  },
  {
    "lsrcodecode": "1005",
    "lsrcodedesc": "เขตบางเขน"
  },
  {
    "lsrcodecode": "1006",
    "lsrcodedesc": "เขตบางกะปิ"
  },
  {
    "lsrcodecode": "1007",
    "lsrcodedesc": "เขตปทุมวัน"
  },
  {
    "lsrcodecode": "1008",
    "lsrcodedesc": "เขตป้อมปราบศัตรูพ่าย"
  },
  {
    "lsrcodecode": "1009",
    "lsrcodedesc": "เขตพระโขนง"
  },
  {
    "lsrcodecode": "1010",
    "lsrcodedesc": "เขตมีนบุรี"
  },
  {
    "lsrcodecode": "1011",
    "lsrcodedesc": "เขตลาดกระบัง"
  },
  {
    "lsrcodecode": "1012",
    "lsrcodedesc": "เขตยานนาวา"
  },
  {
    "lsrcodecode": "1013",
    "lsrcodedesc": "เขตสัมพันธวงศ์"
  },
  {
    "lsrcodecode": "1014",
    "lsrcodedesc": "เขตพญาไท"
  },
  {
    "lsrcodecode": "1015",
    "l

In [10]:
population_statistics_province_result = get_population_statistics_province_view(6412,6412)
print(len(population_statistics_province_result))

[
  {
    "lstrLevel": 0,
    "lsregion": 0,
    "lscc": 10,
    "lsccDesc": "กรุงเทพมหานคร",
    "lsrcode": " ",
    "lsrcodeDesc": " ",
    "lsaa": 0,
    "lsaaDesc": " ",
    "lstt": 0,
    "lsttDesc": " ",
    "lsmm": 0,
    "lsmmDesc": " ",
    "lsyymm": 6412,
    "lssumtotMale": 2592292,
    "lssumtotFemale": 2935702,
    "lssumtotTot": 5527994
  },
  {
    "lstrLevel": 0,
    "lsregion": 0,
    "lscc": 11,
    "lsccDesc": "จังหวัดสมุทรปราการ",
    "lsrcode": " ",
    "lsrcodeDesc": " ",
    "lsaa": 0,
    "lsaaDesc": " ",
    "lstt": 0,
    "lsttDesc": " ",
    "lsmm": 0,
    "lsmmDesc": " ",
    "lsyymm": 6412,
    "lssumtotMale": 645884,
    "lssumtotFemale": 710565,
    "lssumtotTot": 1356449
  },
  {
    "lstrLevel": 0,
    "lsregion": 0,
    "lscc": 12,
    "lsccDesc": "จังหวัดนนทบุรี",
    "lsrcode": " ",
    "lsrcodeDesc": " ",
    "lsaa": 0,
    "lsaaDesc": " ",
    "lstt": 0,
    "lsttDesc": " ",
    "lsmm": 0,
    "lsmmDesc": " ",
    "lsyymm": 6412,
    "lssumtotMale"

In [12]:
import pandas as pd
data = population_statistics_province_result
df = pd.DataFrame(data)
df = df.rename(columns={
    "lsccDesc": "province",
    "lssumtotMale": "male",
    "lssumtotFemale": "female",
    "lssumtotTot": "total"
})

df = df[["province", "male", "female", "total"]]
df.head(5)

,province,male,female,total
0,กรุงเทพมหานคร,2592292,2935702,5527994
1,จังหวัดสมุทรปราการ,645884,710565,1356449
2,จังหวัดนนทบุรี,599167,689470,1288637
3,จังหวัดปทุมธานี,563851,626209,1190060
4,จังหวัดพระนครศรีอยุธยา,394024,426488,820512


In [24]:
  # การแสดง
  # 44 = ผลสำนักทะเบียน/อำเภอ
  # 45 = ตำบล / แขวง
  # 6501 = มกราคม 2565
  # 10 = กรุงเทพ
  # 1001 = เขตพระนคร
  # 100106 = แขวงเสาชิงช้า
data = get_population_statistics_district_view(6501,6512,10,1001)
df = pd.DataFrame(data)
df = df.rename(columns={
    "lsttDesc": "district",
    "lssumtotMale": "male",
    "lssumtotFemale": "female",
    "lssumtotTot": "total"
})

df = df[["district", "male", "female", "total"]]
df.head(5)

,district,male,female,total
0,พระบรมมหาราชวัง,1965,1326,3291
1,วังบูรพาภิรมย์,4567,4534,9101
2,วัดราชบพิธ,1329,1634,2963
3,สำราญราษฎร์,1442,1378,2820
4,ศาลเจ้าพ่อเสือ,1215,1636,2851


In [27]:
  # การแสดง
  # 44 = ผลสำนักทะเบียน/อำเภอ
  # 45 = ตำบล / แขวง
  # 6501 = มกราคม 2565
  # 10 = กรุงเทพ
  # 1001 = เขตพระนคร
  # 10010100 = แขวงพระบรมมหาราชวัง
data = get_population_statistics_subdistrict_view(6512,6512,10,1001,10010100)
df = pd.DataFrame(data)
df = df.rename(columns={
    "lsttDesc": "subdistrict",
    "lssumtotMale": "male",
    "lssumtotFemale": "female",
    "lssumtotTot": "total"
})

df = df[["subdistrict", "male", "female", "total"]]
df.head(5)

,subdistrict,male,female,total
0,แขวงพระบรมมหาราชวัง,1845,1292,3137
